# 과제. BeautifulSoup을 이용하여 아래의 세 기사의 제목과 언론사를 크롤링해주세요!
* for문을 이용하여 url 각각의 기사제목과 언론사를 가져와주세요
* 가져온 정보를 아래와 같이 DataFrame으로 만들어 주세요

|title|company|url|
|------|---|---|
|13일부터 마스크 착용 의무화..한 달 계도 후 과태료 10만 원|YTN|https://news.v.daum.net/v/20201004215700006|
|"사망 10대와 같은 곳서 같은 백신 접종한 32명, 이상반응 없어"|연합뉴스|https://news.v.daum.net/v/20201020153505519|
|지하수에 사는 '골룸 가물치'야, 넌 어디서 왔니|한겨레|https://news.v.daum.net/v/20201020153609574|



# 현주's 과제

In [140]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bs

#### URL의 끝부분을 f스트링으로 바꿔가면서 가져올 수 있게 리스트화 해주자
* 이 예제 말고 검색어, 페이지 넘버 등등으로 바꿔가며 응용도 가능

In [32]:
crawl_url = ["https://news.v.daum.net/v/20201004215700006", "https://news.v.daum.net/v/20201020153505519", "https://news.v.daum.net/v/20201020153609574"]

# 끝의 넘버 값 리스트화 & 기본이 될 URL 만들기

number = ['04215700006','20153505519','20153609574']
base_url = 'https://news.v.daum.net/v/202010{}' #format을 쓰면 여기에 위의 리스트 값들이 들어가겠다

#### 중첩 for문을 써서 기사 타이틀 가져오기

In [112]:
for n in number:
    # 가져올 URL은 포맷팅하는 거니까 괄호 안에 아래와 같이 써줬음
    res = requests.get(base_url.format(n))
    soup = bs(res.content, 'html.parser')
    h3s = soup.select('#cSub > div > h3')
    # 리스트가 각 URL당 하나씩 3개가 반환되므로 각 리스트의 제일 첫 번째 인덱스를 불러와야 제목만 가져옴
    for i in range(0,1):
        print(h3s[i].text)

13일부터 마스크 착용 의무화..한 달 계도 후 과태료 10만 원
"사망 10대와 같은 곳서 같은 백신 접종한 32명, 이상반응 없어"
지하수에 사는 '골룸 가물치'야, 넌 어디서 왔니


#### 언론사를 가져오려면 text를 쓰면 안되는게, 기사제목은 태그 내의 텍스트지만 언론사는 alt라는 속성에 들어있음. 따라서 get 써야함
* for는 반복문! 따라서 기사의 경우 index를 활용해서 세 기사제목을 가져왔고
* 언론사의 경우 company에서 alt라는 속성을 가진 것은 언론사이므로 세개 다 끌어올 수 있는 것

In [129]:
for n in number:
    url = base_url.format(n)
    res = requests.get(url)
    soup = bs(res.content, 'html.parser')
    h3s = soup.select('#cSub > div > h3')
    company = soup.select('#cSub > div > em > a > img')
    for i in range(0,1):
        print (h3s[i].text)
    # 처음에 c in range로 위랑 똑같이 하려고 했으나 실패. 속성값을 끌어와야하므로 get을 쓰고 임의의 c 변수를 생성해준 것!
    for c in company:
        print (c.get('alt'))

13일부터 마스크 착용 의무화..한 달 계도 후 과태료 10만 원
YTN
"사망 10대와 같은 곳서 같은 백신 접종한 32명, 이상반응 없어"
연합뉴스
지하수에 사는 '골룸 가물치'야, 넌 어디서 왔니
한겨레


### 데이터프레임에 담아주기
* 리스트에 기사제목과 언론사를 담아주면 데이터프레임을 만들 수 있다!

In [132]:
# 기사 제목과 언론사를 담을 빈 리스트를 만들어준다

titles = []
companys = []

for n in number:
    url = base_url.format(n)
    res = requests.get(url)
    soup = bs(res.content, 'html.parser')
    h3s = soup.select('#cSub > div > h3')
    company = soup.select('#cSub > div > em > a > img')
    # append를 써서 for 문에서 도출되는 내용들을 리스트에 추가한다
    for i in range(0,1):
        titles.append(h3s[i].text)
    for c in company:
        companys.append(c.get('alt'))

In [133]:
titles

['13일부터 마스크 착용 의무화..한 달 계도 후 과태료 10만 원',
 '"사망 10대와 같은 곳서 같은 백신 접종한 32명, 이상반응 없어"',
 "지하수에 사는 '골룸 가물치'야, 넌 어디서 왔니"]

In [134]:
companys

['YTN', '연합뉴스', '한겨레']

In [139]:
# 아래 새로 배운 내용 중 2번 방법으로! 여러 리스트가 있을 때 딕셔너리 형태로 데이터 프레임을 만들어 줄 수 있다

df = pd.DataFrame(
    {'title': titles,
    'company': companys,
    'url': crawl_url})
df

,title,company,url
0,13일부터 마스크 착용 의무화..한 달 계도 후 과태료 10만 원,YTN,https://news.v.daum.net/v/20201004215700006
1,"""사망 10대와 같은 곳서 같은 백신 접종한 32명, 이상반응 없어""",연합뉴스,https://news.v.daum.net/v/20201020153505519
2,"지하수에 사는 '골룸 가물치'야, 넌 어디서 왔니",한겨레,https://news.v.daum.net/v/20201020153609574


#### 숙제끝! 파톺 화이팅♥
**==============================================================================================================**

## 과제 진행하면서 알게 된 내용들

### For문이 아닐 때의 결과
* 맨 마지막 값만 가져온다! <br>
* 처음 가지고 온 값은 그 뒤에 가져온 값으로 **덮어 쓰여지기 때문**에 맨 마지막 값만 남는 것

In [131]:
h3s = soup.select('#cSub > div > h3')
h3s[0].text

"지하수에 사는 '골룸 가물치'야, 넌 어디서 왔니"

### 데이터프레임 형성하는 세 가지 방법
*1. 리스트/어레이/시리즈로 만들기*<br>
*2. 딕셔너리 형태로 만들기*<br>
*3. 딕셔너리의 리스트 형태로 만들기*

#### 1. 리스트/어레이/시리즈 형태

In [174]:
hj_list = [['현주', 165], ['승혁', 178], ['푸', 50], ['피글렛', 30]]
hj_array = np.array(hj_list)

In [175]:
df1 = pd.DataFrame(
    hj_list, 
    columns = ['name', 'height'],
    index = [1,2,3,4])

df2 = pd.DataFrame(hj_array, columns = ['name', 'height'])

#### 2. 리스트가 여러개 있을 때, 딕셔너리

In [176]:
names = ['현주', '승혁', '푸', '피글렛']
height = [165, 178, 50, 30]

In [179]:
df3 = pd.DataFrame({
    'name': names,
    'height': height
})

df4 = pd.DataFrame({
    'name': np.array(names),
    'height': np.array(height)},
    index = [1,2,3,4])

df5 = pd.DataFrame({
    'name': pd.Series(names),
    'height': pd.Series(height)
})

#### 3. 딕셔너리 여러개가 포함된 리스트

In [183]:
hj_dict = [
    {'name': '현주', 'height': 165},
    {'name': '승혁', 'height': 178},
    {'name': '푸', 'height': 50},
    {'name': '피글렛', 'height': 30}
]

In [184]:
df6 = pd.DataFrame(
    hj_dict,
    index = [1,2,3,4])